In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import os
import warnings
from xgboost import XGBClassifier

# Problem 3

In [2]:
PATH = os.chdir('/Users/microwave/AIO_2024/Module_3/Week5')
df = pd.read_csv('Problem 3.csv')
df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,4.468204,26.2,94.3,1.808289,8.2,51,6.7,False,0.000000
1,7,4,oct,tue,4.517431,35.4,669.1,2.041220,18.0,33,0.9,False,0.000000
2,7,4,oct,sat,4.517431,43.7,686.9,2.041220,14.6,33,1.3,False,0.000000
3,8,6,mar,fri,4.529368,33.3,77.5,2.302585,8.3,97,4.0,True,0.000000
4,8,6,mar,sun,4.503137,51.3,102.2,2.360854,11.4,99,1.8,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,aug,sun,4.414010,56.7,665.6,1.064711,27.8,32,2.7,False,2.006871
506,2,4,aug,sun,4.414010,56.7,665.6,1.064711,21.9,71,5.8,False,4.012592
507,7,4,aug,sun,4.414010,56.7,665.6,1.064711,21.2,70,6.7,False,2.498152
508,1,4,aug,sat,4.558079,146.0,614.7,2.509599,25.6,42,4.0,False,0.000000


In [3]:
categorical_cols = df.select_dtypes(include = ['bool','object']).columns.to_list()

for col_name in categorical_cols:
    n_categories = df[col_name].nunique()
    print(f'Number of categories in {col_name}: {n_categories}')

ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder.fit_transform(df[categorical_cols])

encoded_categorical_df = pd.DataFrame(encoded_categorical_cols,columns = categorical_cols)

numerical_df = df.drop(categorical_cols,axis = 1)
encoded_df = pd.concat([numerical_df,encoded_categorical_df],axis = 1)


Number of categories in month: 12
Number of categories in day: 7
Number of categories in rain: 2


In [4]:
X = encoded_df.drop(columns = ['area'])
y = encoded_df['area']

SEED = 7
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=SEED)


In [5]:
from sklearn.model_selection import GridSearchCV


lr = 0.01
N_ESTIMATORS = 102
MAX_DEPTH = 3 

params = {'objective':['reg:squarederror'],
          'learning_rate' : [0.01,0.02,0.03,0.04],
          'max_depth' : range(3,7),
          'n_estimators': range(100,150,1),
          'gamma':range(0,3)}

xgb = xgb.XGBRegressor()
xgb_grid = GridSearchCV(xgb,params,cv=2,n_jobs=3)
xgb_grid.fit(X_train,y_train)

print(f'Best params for XGB Regressor: {xgb_grid.best_params_}')
print(f'Best score for XGB Regressor: {xgb_grid.best_score_}')


/Users/microwave/opt/anaconda3/lib/python3.9/site-packages/numpy/ma/core.py:2815: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best params for XGB Regressor: {'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 139, 'objective': 'reg:squarederror'}
Best score for XGB Regressor: -0.0061967903336567765


In [6]:
y_pred = xgb_grid.predict(X_test)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f'Mean absolute error: {mae}')
print(f'Mean squared error: {mse}')

Mean absolute error: 1.1593882730377256
Mean squared error: 1.951430694908722


# Problem 1

In [7]:
df = pd.read_csv('Problem 4.csv')
df.head(5)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [8]:
X,y = df.iloc[:,:-1], df.iloc[:,-1]

X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=SEED)

In [9]:
params = {
    'objective': ['binary:logistic'],
    'learning_rate': [0.01, 0.02, 0.03, 0.04],
    'max_depth': range(3, 7),
    'n_estimators': range(100, 150, 1),
    'gamma': range(0, 3)
}

xgb_clf = XGBClassifier()
xgb_grid_clf = GridSearchCV(xgb_clf, params, cv=2, n_jobs=3)
xgb_grid_clf.fit(X_train, y_train)

print(f'Best params for XGB Classifier: {xgb_grid_clf.best_params_}')
print(f'Best score for XGB Classifier: {xgb_grid_clf.best_score_}')


/Users/microwave/opt/anaconda3/lib/python3.9/site-packages/numpy/ma/core.py:2815: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best params for XGB Classifier: {'gamma': 2, 'learning_rate': 0.02, 'max_depth': 3, 'n_estimators': 103, 'objective': 'binary:logistic'}
Best score for XGB Classifier: 0.9435483870967742


In [10]:
y_pred = xgb_grid_clf.predict(X_test)
y_train_pred = xgb_grid_clf.predict(X_train)

In [11]:
train_acc = accuracy_score(y_train, y_train_pred,)
test_acc = accuracy_score(y_test,y_pred)

print(f'Train accuracy: {train_acc}')
print(f'Test accuracy {test_acc}')


Train accuracy: 1.0
Test accuracy 0.9629629629629629
